# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [57]:
# Dada una palabra (query), se buscará en el corpus y se retorna los docuemntos en los que aparece
def buscar_query(nombre_archivo, query):
    with open(nombre_archivo, 'r', encoding='utf-8') as corpus:
        documentos = corpus.readlines()
    
    resultados = []
    indice_doc_contenedor = 0

    for line in documentos:
        if query.lower() in line.lower():
            resultados.append(f"Documento {indice_doc_contenedor}: {line.strip()}")
        indice_doc_contenedor += 1

    if resultados:
        print(resultados)
    else:
        print('No se encontro el query en el corpus')

In [58]:
nombre_archivo = '01_corpus_turismo.txt'
query = "turistas"
buscar_query(nombre_archivo, query)

['Documento 3: En Baños de Agua Santa, los turistas disfrutan de deportes de aventura como rafting y canopy.', 'Documento 6: Cuenca deslumbra a los turistas con su arquitectura colonial y su gastronomía tradicional.', 'Documento 12: El Parque Nacional Cajas invita a los turistas a caminatas entre lagunas y páramos a gran altitud.', 'Documento 13: Los turistas disfrutan en el feriado de las fiestas locales y de la gastronomía típica de cada región.']


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [59]:
# Construcción de un diccionario índice invertido
def construir_indice_invertido(nombre_archivo):
    with open(nombre_archivo, 'r', encoding='utf-8') as corpus:
        documentos = corpus.readlines()
    
    indice_invertido = {}

    for i, documento in enumerate(documentos):
        palabras = documento.lower().strip().split()
        
        for palabra in palabras:
            palabra = palabra.strip('.,;:!?()"\'')
            if palabra:
                if palabra not in indice_invertido:
                    indice_invertido[palabra] = set()
                indice_invertido[palabra].add(i)

    return indice_invertido, documentos

In [60]:
# Funcion de busqueda que utiliza el indice invertido
def buscar_con_indice(indice_invertido, documentos, query):
    query = query.lower().strip('.,;:!?()"\'')
    if query in indice_invertido:
        indices_documentos = indice_invertido[query]
        resultados=[]
        for i in indices_documentos:
            resultados.append(f"Documento {i}: {documentos[i].strip()}")

        print(resultados)
    else:
        print('No se encontró el query en el corpus')

In [61]:
nombre_archivo = '01_corpus_turismo.txt'
indice_invertido, documentos = construir_indice_invertido(nombre_archivo)

buscar_con_indice(indice_invertido, documentos, 'turistas')

['Documento 3: En Baños de Agua Santa, los turistas disfrutan de deportes de aventura como rafting y canopy.', 'Documento 12: El Parque Nacional Cajas invita a los turistas a caminatas entre lagunas y páramos a gran altitud.', 'Documento 13: Los turistas disfrutan en el feriado de las fiestas locales y de la gastronomía típica de cada región.', 'Documento 6: Cuenca deslumbra a los turistas con su arquitectura colonial y su gastronomía tradicional.']


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [62]:
import time

In [63]:
def medir_tiempo_busqueda_lineal(nombre_archivo, querys):
    tiempos = []
    for query in querys:
        inicio = time.time()
        buscar_query(nombre_archivo, query)
        fin = time.time()
        tiempos.append(fin - inicio)
    return tiempos

def medir_tiempo_busqueda_indice_invertido(indice_invertido, documentos, querys):
    tiempos = []
    for query in querys:
        inicio = time.time()
        buscar_con_indice(indice_invertido, documentos, query)
        fin = time.time()
        tiempos.append(fin - inicio)
    return tiempos

In [64]:
corpus_pequenio = '01_corpus_turismo.txt'
corpus_grande = '01_corpus_turismo_500.txt'

querys = ["quito", "montañita", "feriado", "playas", "aventura", "galápagos"]

In [65]:
# -------- Corpus pequeño --------

tiempos_lineal_pequenio = medir_tiempo_busqueda_lineal(corpus_pequenio, querys)

indice_pequenio, documentos_pequenio = construir_indice_invertido(corpus_pequenio)
tiempos_indice_pequenio = medir_tiempo_busqueda_indice_invertido(indice_pequenio, documentos_pequenio, querys)

['Documento 2: La ciudad de Quito cuenta con un centro histórico declarado Patrimonio de la Humanidad por la UNESCO.']
['Documento 4: La Ruta del Spondylus conecta playas famosas como Montañita, Salinas y Puerto López.', 'Documento 15: Montañita se llena de surfistas y viajeros jóvenes durante los feriados nacionales.']
['Documento 11: En el feriado, muchos ecuatorianos visitan la Amazonía para conocer comunidades indígenas y cascadas.', 'Documento 13: Los turistas disfrutan en el feriado de las fiestas locales y de la gastronomía típica de cada región.', 'Documento 15: Montañita se llena de surfistas y viajeros jóvenes durante los feriados nacionales.']
['Documento 0: Ecuador es un país megadiverso, con playas, montañas y selvas ideales para el turismo ecológico.', 'Documento 4: La Ruta del Spondylus conecta playas famosas como Montañita, Salinas y Puerto López.', 'Documento 9: Guayaquil sorprende con su Malecón 2000, su vida nocturna y su cercanía a las playas del Pacífico.']
['Docum

In [66]:
# -------- Corpus grande --------
# Búsqueda lineal
tiempos_lineal_grande = medir_tiempo_busqueda_lineal(corpus_grande, querys)

# Índice invertido
indice_grande, documentos_grande = construir_indice_invertido(corpus_grande)
tiempos_indice_grande = medir_tiempo_busqueda_indice_invertido(indice_grande, documentos_grande, querys)

['Documento 3: Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para surf.', 'Documento 15: Quito tiene un centro histórico Patrimonio de la Humanidad Ideal para el próximo feriado.', 'Documento 16: Quito tiene un centro histórico Patrimonio de la Humanidad Un lugar sorprendente para visitar.', 'Documento 38: Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para rafting.', 'Documento 42: Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para senderismo.', 'Documento 72: Quito tiene un centro histórico Patrimonio de la Humanidad Ideal para el próximo feriado.', 'Documento 74: Quito tiene un centro histórico Patrimonio de la Humanidad Ideal para el próximo feriado.', 'Documento 88: Quito tiene un centro histórico Patrimonio de la Humanidad', 'Documento 103: Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para canopy.', 'Documento 135: Quito tiene un centro histórico Patrimonio de la Humanidad Perfecto para canopy

In [67]:
def mostrar_tabla(querys, tiempos_lineal, tiempos_indice_invertido, corpus_nombre):
    print(f"\nResultados para {corpus_nombre}:")
    print(f"{'Palabra':<15} {'Lineal (s)':<15} {'Índice (s)':<15}")
    print("-" * 45)
    for palabra, t_lineal, t_indice in zip(querys, tiempos_lineal, tiempos_indice_invertido):
        print(f"{palabra:<15} {t_lineal:<15.6f} {t_indice:<15.6f}")

# Mostrar resultados
mostrar_tabla(querys, tiempos_lineal_pequenio, tiempos_indice_pequenio, "Corpus pequeño")
mostrar_tabla(querys, tiempos_lineal_grande, tiempos_indice_grande, "Corpus grande")



Resultados para Corpus pequeño:
Palabra         Lineal (s)      Índice (s)     
---------------------------------------------
quito           0.001001        0.000000       
montañita       0.001516        0.000000       
feriado         0.000000        0.000000       
playas          0.000000        0.000000       
aventura        0.001011        0.000000       
galápagos       0.000000        0.000000       

Resultados para Corpus grande:
Palabra         Lineal (s)      Índice (s)     
---------------------------------------------
quito           0.002000        0.000000       
montañita       0.002000        0.000000       
feriado         0.001996        0.000000       
playas          0.000000        0.000000       
aventura        0.001088        0.000000       
galápagos       0.000998        0.000000       


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [68]:
# Modificación de índice para que no difiera de mayusculas ni minusculas ni separadores
# Funcion de busqueda que utiliza el indice invertido
def buscar_con_indice(indice_invertido, documentos, query):
    query = query.lower().strip('.,;:!?()"\'')
    if query in indice_invertido:
        indices_documentos = indice_invertido[query]
        resultados=[]
        for i in indices_documentos:
            resultados.append(f"Documento {i}: {documentos[i].strip()}")

        print(resultados)
    else:
        print('No se encontró el query en el corpus')

In [ ]:
def buscar_multi_terminos(indice_invertido, documentos, querys):
    query = querys.lower().strip().split()

    palabras = []
    for p in query:
        if p:
            palabra = p.strip('.,;:!?()"\'')
            palabras.append(palabra)

    documentos_comunes = indice_invertido.get(palabras[0], set()).copy()

    for palabra in palabras[1:]:
        documentos_comunes = documentos_comunes & indice_invertido.get(palabra, set())

    resultados = []
    for i in documentos_comunes:
        texto = documentos[i].strip()
        resultados.append(f"Documento {i}: {texto}")

    return resultados

In [91]:
buscar_multi_terminos(indice_invertido, documentos, "playas turismo")

[]

In [92]:
def calcular_speedup(nombre_archivo, indice_invertido, documentos, query):
    # ----- Tiempo de búsqueda lineal -----
    inicio_lineal = time.time()
    buscar_query(nombre_archivo, query)
    fin_lineal = time.time()
    tiempo_lineal = fin_lineal - inicio_lineal

    # ----- Tiempo de búsqueda con índice invertido -----
    inicio_indice = time.time()
    buscar_multi_terminos(indice_invertido, documentos, query)
    fin_indice = time.time()
    tiempo_indice = fin_indice - inicio_indice

    # ----- Calcular speedup -----
    if tiempo_indice > 0:
        speedup = tiempo_lineal / tiempo_indice
    else:
        if tiempo_indice < 1e-6:
            print("Índice invertido extremadamente rápido (casi 0 segundos)")
            speedup = float('inf')


    # ----- Mostrar resultados -----
    print(f"Tiempo búsqueda lineal: {tiempo_lineal:.6f} segundos")
    print(f"Tiempo búsqueda con índice invertido: {tiempo_indice:.6f} segundos")
    print(f"Speedup: {speedup:.2f} veces más rápido")

    return speedup

In [93]:
nombre_archivo = '01_corpus_turismo_500.txt'
indice_invertido, documentos = construir_indice_invertido(nombre_archivo)
calcular_speedup(nombre_archivo, indice_invertido, documentos, "2000")

['Documento 12: Guayaquil sorprende con su Malecón 2000 y vida nocturna Un lugar espectacular para visitar.', 'Documento 14: Guayaquil sorprende con su Malecón 2000 y vida nocturna Perfecto para senderismo.', 'Documento 20: Guayaquil sorprende con su Malecón 2000 y vida nocturna', 'Documento 54: Guayaquil sorprende con su Malecón 2000 y vida nocturna', 'Documento 70: Guayaquil sorprende con su Malecón 2000 y vida nocturna Una experiencia inolvidable.', 'Documento 79: Guayaquil sorprende con su Malecón 2000 y vida nocturna Ideal para el próximo feriado.', 'Documento 104: Guayaquil sorprende con su Malecón 2000 y vida nocturna Un lugar espectacular para visitar.', 'Documento 115: Guayaquil sorprende con su Malecón 2000 y vida nocturna Ideal para el próximo feriado.', 'Documento 122: Guayaquil sorprende con su Malecón 2000 y vida nocturna Ideal para el próximo feriado.', 'Documento 130: Guayaquil sorprende con su Malecón 2000 y vida nocturna', 'Documento 146: Guayaquil sorprende con su Ma

inf